컴퓨터의 처리 능력 : 숫자>>텍스트 → 자연어처리에는 텍스트를 숫자로 바꾸는 기법 존재 : 첫 단계=맵핑시키는 전처리 작업

ex) 텍스트에 단어 5,000개 : 5,000개의 단어들 각각에 1번부터 5,000번까지 단어와 맵핑되는 고유한 정수, 인덱스를 부여  
가령, book은 150번, dog는 171번

인덱스 부여 방법 : 랜덤, 빈도수 기준 정렬 후 부여, ...

# 1. 정수 인코딩(Integer Encoding)
빈도수 순으로 정렬한 단어 집합(vocabulary)을 만들고, 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여

#### 1) dictionary 사용하기

In [1]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

raw_text= "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [2]:
# 문장 토큰화
sentences=sent_tokenize(raw_text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [3]:
vocab={}
preprocessed_sentences=[]
stop_words=set(stopwords.words('english'))

for sentence in sentences:
    tokenized_sentence=word_tokenize(sentence)
    result=[]
    
    for word in tokenized_sentence:
        word=word.lower() # 모든 단어 소문자화 → 단어 개수 축소
        if word not in stop_words:
            if len(word)>2: # 단어 길이가 2 이하인 단어 제거
                result.append(word)
                if word not in vocab:
                    vocab[word]=0
                vocab[word]+=1
    preprocessed_sentences.append(result)
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [4]:
# 빈도수 확인
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [5]:
# 'barber' 라는 단어 빈도수 출력
print(vocab['barber'])

8


In [6]:
# 빈도수 내림치순 정렬
vocab_sorted=sorted(vocab.items(), key=lambda x:x[1], reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [7]:
# 정수 인코딩
word_to_index={}
i=0
for (word, frequency) in vocab_sorted:
    if frequency>1: # 빈도수 작은 단어 제외 : 자연어 처리에서 의미 가지지 않음
        i+=1
        word_to_index[word]=i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [8]:
# 빈도수 상위 n개의 단어만 사용
vocab_size=5

# 인덱스가 5 초과인 단어 제거
word_frequency=[word for word, index in word_to_index.items() if index>=vocab_size+1]

# 해당 단어에 대한 인덱스 정보를 삭제
for w in word_frequency:
    del word_to_index[w]
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


인코딩

- 첫번째 문장 ['barber', 'person'] → [1, 5]  
- 두번째 문장 ['barber', 'good', 'person'] → 집합에 존재하지 않는 단어 : Out-Of-Vocabulary : **OOV 문제** : word_to_index에 'OOV'란 단어를 새롭게 추가하고, 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩

In [9]:
word_to_index['OOV']=len(word_to_index)+1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [10]:
encoded_sentences=[]
for sentence in preprocessed_sentences:
    encoded_sentence=[]
    for word in sentence:
        try:
            encoded_sentence.append(word_to_index[word])
        except KeyError:
            encoded_sentence.append(word_to_index['OOV'])
    encoded_sentences.append(encoded_sentence)
print(encoded_sentences)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


#### 2) Counter 사용하기

In [11]:
from collections import Counter

In [12]:
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [13]:
all_word_list=sum(preprocessed_sentences, [])
print(all_word_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [14]:
vocab=Counter(all_word_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [15]:
vocab_size=5
vocab=vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [16]:
word_to_index={}
index=1
for (word, frequency) in vocab:
    word_to_index[word]=index
    index+=1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


#### 3) NLTK의 FreqDist 사용하기

In [17]:
from nltk import FreqDist
import numpy as np

In [18]:
vocab=FreqDist(np.hstack(preprocessed_sentences))

In [19]:
print(vocab['barber'])

8


In [20]:
vocab_size=5
vocab=vocab.most_common(vocab_size)
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [21]:
word_to_index={word[0] : index+1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


# 2. 케라스(Keras)의 텍스트 전처리
Keras는 기본적인 전처리를 위한 도구 제공

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [23]:
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [24]:
tokenizer=Tokenizer()

# fit_on_texts(corpus) : corpus의 빈도수 기준으로 단어 집합 생성
tokenizer.fit_on_texts(preprocessed_sentences)

In [25]:
# word_index : 각 단어 인덱스 부여
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [26]:
# word_counts : 각 단어가 카운트를 수행하였을 때 몇 개였는지 확인
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [27]:
# text_to_sentences(corpus) : 각 단어를 이미 정해진 인덱스로 변환
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [28]:
vocab_size=5
tokenizer=Tokenizer(num_words=vocab_size+1)
tokenizer.fit_on_texts(preprocessed_sentences)

num_words=vocab_size의 적용은 text_to_sequences에서만 적용인 된다.

In [29]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [30]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [31]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


word_index, word_counts에도 num_words 기준을 적용해보자.

In [32]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

In [33]:
vocab_size=5
words_frequency=[word for word, index in tokenizer.word_index.items() if index>vocab_size]

for word in words_frequency:
    del tokenizer.word_index[word]
    del tokenizer.word_counts[word]

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


Keras의 토크나이저는 기본적으로 OOV에 대해서 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징이 있다. 단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용한다.

In [34]:
vocab_size=5
tokenizer=Tokenizer(num_words=vocab_size+2, oov_token='OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

In [35]:
print("단어 OOV의 인덱스 : {}".format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 : 1


In [36]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]
